Ноутбук подготовлен на основе материалов курса NLP Школы анализа данных: https://github.com/yandexdataschool/nlp_course

# Эмбеддинги

In [2]:
!pip install gensim

In [3]:
import gensim.downloader as api
model = api.load('glove-twitter-100')

[==================================================] 100.0% 387.1/387.1MB downloaded


C:\Users\Yury\Anaconda3\lib\site-packages\smart_open\smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [4]:
model.most_similar(positive=["king", "woman"], negative=["man"])

[('queen', 0.7052316069602966),
 ('prince', 0.6666139364242554),
 ('mother', 0.6436765193939209),
 ('royal', 0.6417250633239746),
 ('father', 0.5952690243721008),
 ('african', 0.5883978009223938),
 ('princess', 0.588217556476593),
 ('called', 0.5842776298522949),
 ('meets', 0.584027886390686),
 ('american', 0.5815179944038391)]

In [5]:
words = sorted(model.vocab.keys(), 
               key=lambda word: model.vocab[word].count,
               reverse=True)[:1000]

print(words[::100])

['<user>', '_', 'please', 'apa', 'justin', 'text', 'hari', 'playing', 'once', 'sei']


In [6]:
import numpy as np

In [7]:
a = [2,3,4]
[item * 2 for item in a]

[4, 6, 8]

In [8]:
word_vectors = np.array([model.get_vector(word)
               for word in words])

## Визуализация эмбеддингов слов с помощью t-SNE

In [9]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    if isinstance(color, str): color = [color] * len(x)
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [10]:
from sklearn.manifold import TSNE

# map word vectors onto 2d plane with TSNE. hint: use verbose=100 to see what it's doing.

word_tsne = TSNE(n_components=2, verbose=100).fit_transform(word_vectors)

word_tsne = (word_tsne - np.average(
    word_tsne, axis=0).reshape(1, 2)) / np.var(
    word_tsne, axis=0).reshape(1, 2) ** 0.5

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1000 samples in 0.010s...
[t-SNE] Computed neighbors for 1000 samples in 0.310s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1000
[t-SNE] Mean sigma: 1.716134
[t-SNE] Computed conditional probabilities in 0.062s
[t-SNE] Iteration 50: error = 68.4813385, gradient norm = 0.3074562 (50 iterations in 31.367s)
[t-SNE] Iteration 100: error = 69.4811707, gradient norm = 0.2839635 (50 iterations in 31.639s)
[t-SNE] Iteration 150: error = 70.3053284, gradient norm = 0.2752785 (50 iterations in 32.431s)
[t-SNE] Iteration 200: error = 69.8299484, gradient norm = 0.2863117 (50 iterations in 32.177s)
[t-SNE] Iteration 250: error = 69.9879379, gradient norm = 0.2566306 (50 iterations in 30.481s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 69.987938
[t-SNE] Iteration 300: error = 1.2056882, gradient norm = 0.0032409 (50 iterations in 21.851s)
[t-SNE] Iteration 350: error = 1.1167022, gradient norm = 0

In [11]:
draw_vectors(word_tsne[:, 0], word_tsne[:, 1], color='green', token=words)

Figure(id='1003', ...)

# Простая вопросно-ответная система

In [14]:
# download the data:
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

"wget" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


In [16]:
import numpy as np

data = list(open("./quora.txt", encoding='utf-8'))
data[50]

"What TV shows or books help you read people's body language?\n"

In [17]:
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()

print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


In [18]:
# data_tok should be a list of lists of tokens for each line in data.

data_tok = [tokenizer.tokenize(piece_of_data.lower())
           for piece_of_data in data]

In [19]:
data_tok[5]

['what', 'can', 'i', 'do', 'to', 'improve', 'my', 'immune', 'system', '?']

In [20]:
def get_phrase_embedding(phrase):
    """
    Convert phrase to a vector by aggregating it's word embeddings. See description above.
    """
    # 1. lowercase phrase
    # 2. tokenize phrase
    # 3. average word vectors for all words in tokenized phrase
    # skip words that are not in model's vocabulary
    # if all words are missing from vocabulary, return zeros
    
    
    words = tokenizer.tokenize(phrase.lower())
    known_vectors = []
    for word in words:
        if word in model.vocab:
            known_vectors.append(model.get_vector(word))
    known_vectors = np.array(known_vectors)
    if known_vectors.size != 0: 
        vector = known_vectors.mean(axis=0)
    else:
        vector = np.zeros([model.vector_size], dtype='float32')

    return vector

In [21]:
# let's only consider ~5k phrases for a first run.
chosen_phrases = data[::len(data) // 1000]

# compute vectors for chosen phrases
phrase_vectors = np.array([get_phrase_embedding(phrase)
                 for phrase in chosen_phrases])

In [22]:
# map vectors into 2d space with pca, tsne or your other method of choice
# don't forget to normalize

phrase_vectors_2d = TSNE(verbose=1000, n_iter=300).fit_transform(phrase_vectors)

phrase_vectors_2d = (phrase_vectors_2d - 
                     np.mean(phrase_vectors_2d, axis=0)) / np.std(phrase_vectors_2d, axis=0)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1001 samples in 0.007s...
[t-SNE] Computed neighbors for 1001 samples in 0.347s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1001
[t-SNE] Computed conditional probabilities for sample 1001 / 1001
[t-SNE] Mean sigma: 0.477874
[t-SNE] Computed conditional probabilities in 0.078s
[t-SNE] Iteration 50: error = 79.2543488, gradient norm = 0.3045631 (50 iterations in 32.673s)
[t-SNE] Iteration 100: error = 79.9931870, gradient norm = 0.2962231 (50 iterations in 35.809s)
[t-SNE] Iteration 150: error = 79.0348892, gradient norm = 0.3103170 (50 iterations in 34.528s)
[t-SNE] Iteration 200: error = 80.6777802, gradient norm = 0.3119746 (50 iterations in 34.600s)
[t-SNE] Iteration 250: error = 81.8536301, gradient norm = 0.3073660 (50 iterations in 35.334s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 81.853630
[t-SNE] Iteration 300: error = 2.0719197, gradient norm = 0.0038935 (50 iterations in 23

In [23]:
draw_vectors(phrase_vectors_2d[:, 0], phrase_vectors_2d[:, 1],
             phrase=[phrase[:50] for phrase in chosen_phrases],
             radius=20,)

Figure(id='1112', ...)

In [24]:
# compute vector embedding for all lines in data
data_vectors = np.array([get_phrase_embedding(l) for l in data])

In [25]:
from sklearn.metrics.pairwise import cosine_distances
def find_nearest(query, k=10):
    """
    given text line (query), return k most similar lines from data, sorted from most to least similar
    similarity should be measured as cosine between query and line embedding vectors
    hint: it's okay to use global variables: data and data_vectors. see also: np.argsort
    """
        
    query_vector = get_phrase_embedding(query)
    
    distances = cosine_distances(data_vectors, query_vector[None, :])
    
    indices = np.argsort(distances[:, 0])[:k]   

    return [data[index] 
            for index in indices]

In [26]:
results = find_nearest(query="How do i enter the matrix?", k=10)

print(''.join(results))

How do I get to the dark web?
What should I do to enter hollywood?
How do I use the Greenify app?
What can I do to save the world?
How do I win this?
How do I think out of the box? How do I learn to think out of the box?
How do I find the 5th dimension?
How do I use the pad in MMA?
How do I estimate the competition?
What do I do to enter the line of event management?



In [27]:
find_nearest(query="How does Trump?", k=10)

['What does Donald Trump think about Israel?\n',
 'What books does Donald Trump like?\n',
 'What does Donald Trump think of India?\n',
 'What does India think of Donald Trump?\n',
 'What does Donald Trump think of China?\n',
 'What does Donald Trump think about Pakistan?\n',
 'What companies does Donald Trump own?\n',
 'What does Dushka Zapata think about Donald Trump?\n',
 'How does it feel to date Ivanka Trump?\n',
 'What does salesforce mean?\n']

In [29]:
find_nearest(query="How old is Bill Gates?\n", k=10)

['Why is Bill Gates so rich?\n',
 'Why and how is Bill Gates so rich?\n',
 'What made Bill Gates rich?\n',
 'What kind of phone does Bill Gates have?\n',
 'Why is Steve Jobs considered a genius where else Bill gates is not?\n',
 'Where is Bill Stachour?\n',
 'How old is trump?\n',
 'How does Bill Gates spend his days?\n',
 'How does Bill Gates spend his days now?\n',
 'How did Bill Gates choose the name Microsoft?\n']